<center>
    <img src='images/small-uai.jpeg'style="width: 300px;">
</center>

# Laboratorio S02: Clasificación Lineal y Redes Neuronales

## _Deep learning_
   
<center>
    <img src='images/clasificacion-lineal.png'style="width: 400px;">
    <sub><sup>http://eenube.com/images/sampledata/moon_reg_log.png</sup></sub> 
</center>

**Profesor**: Dr. Juan Bekios Calfa

**Grado**: MIA

# Laboratorio 02: Clasificación Lineal y Redes Neuronales

Objetivo:

1.   Implementar una regresión logística
2.   Implementar una red neuronal simple como **regresor**



## 1. Regresión logística

Clasificación binaria utilizando regresión logística. Implemntado en Pytorch.



In [2]:
# Librerías

import torch.nn as nn
import torch
import numpy as np
from sklearn import datasets # Base de datos que vamos a utilizar
from sklearn.preprocessing import StandardScaler # Escalar los datos de entrada
from sklearn.model_selection import train_test_split # Crear el conjunto de entrenamiento y pruebas

In [5]:
# 0) Preparar los datos
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_samples, n_features = X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# Escalar
sc = StandardScaler()
X_train = sc.fit_transform(X_train) # Entrenar
#X_train = sc.transform(X_train) # Conjunto de entrenamiento va quedar escalado
X_test = sc.transform(X_test) # Conjunto de prueba escalado

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
      

In [7]:
# 1) Modelo
class Model(nn.Module):
  # Declarar las partes del modelo
  def __init__(self, n_input_features):
    super(Model, self).__init__()
    self.linear = nn.Linear(n_input_features, 1)

  # Calcular la estimacion (x <- vector de entrada)
  def forward(self,x):
    y_pred = torch.sigmoid(self.linear(x))
    return y_pred

model = Model(n_features)

# 2) Loss y optimizar
num_epochs = 100
learning_rate = 0.02
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)



In [10]:
# 3) Ciclo entrenamiento
for epoch in range(num_epochs):
  # Forward 
  y_pred = model(X_train)
  loss = criterion(y_pred, y_train) # Error

  # Calcular gradientes
  loss.backward()
  optimizer.step()

  # Cero los gradientes
  optimizer.zero_grad()

  if (epoch+1) % 10 == 0:
    print(f'epoch: {epoch+1},, loss = {loss.item():.4f}')

# 4) Comparar con el conjunto de pruebas
with torch.no_grad():
  y_predicted = model(X_test)
  y_predicted_cls = y_predicted.round()
  acc = y_predicted_cls.eq(y_test).sum() /float(y_test.shape[0])
  print(f'Tasa de acierto: {acc.item():.4f}')

epoch: 10,, loss = 0.1285
epoch: 20,, loss = 0.1259
epoch: 30,, loss = 0.1236
epoch: 40,, loss = 0.1214
epoch: 50,, loss = 0.1193
epoch: 60,, loss = 0.1173
epoch: 70,, loss = 0.1155
epoch: 80,, loss = 0.1138
epoch: 90,, loss = 0.1121
epoch: 100,, loss = 0.1106
Tasa de acierto: 0.9035


## 2. Red neuronal poco profunda

Regresión utilizando una red neuronal poca profunda. Implementada en Pytorch.

El problema tiene solo **un atributo de entrada** y una **etiquta de salida**.

### Librerías

In [12]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

### Definir el modelo neuronal


In [46]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(1, 2, bias=True)
    self.fc2 = nn.Linear(2, 1, bias=True)

  # Prediccion
  def forward(self, x):
    x = self.fc2(F.relu(self.fc1(x)))
    return x

### Parámetros de optimización

In [47]:
net = Net().cuda()

### Definir una base de datos hipotética

In [48]:
print(net)

Net(
  (fc1): Linear(in_features=1, out_features=2, bias=True)
  (fc2): Linear(in_features=2, out_features=1, bias=True)
)


### Entrenamiento de la red

In [49]:
print(list(net.parameters()))

[Parameter containing:
tensor([[ 0.4838],
        [-0.1446]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.5787,  0.1759], device='cuda:0', requires_grad=True), Parameter containing:
tensor([[-0.0662,  0.1834]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.1189], device='cuda:0', requires_grad=True)]


In [18]:
input = Variable(torch.randn(1,1,1)).cuda()
print(input)

tensor([[[1.3158]]], device='cuda:0')
tensor([[[-1.0171]]])


In [20]:
# Dejar los gradientes en cero
net.zero_grad()

In [21]:
out = net(input)
print(out)

tensor([[[-0.6601]]], device='cuda:0', grad_fn=<AddBackward0>)


# Entrenar

In [22]:
import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.4)
criterion = nn.MSELoss()

In [23]:
data = [(1,3), (2, 6), (3,9), (4, 12), (5, 15), (6,18)]

In [24]:
for epoch in range(100):
  for i, data2 in enumerate(data):
    X, Y = iter(data2)
    X, Y = Variable(torch.FloatTensor([X]), requires_grad=True).cuda(), Variable(torch.FloatTensor([Y]), requires_grad=False).cuda()
    optimizer.zero_grad()
    y_pred =  net(X)
    output = criterion(y_pred, Y)
    output.backward()
    optimizer.step()

  if (epoch % 20 == 0.0):
    print('Epoch {} - loss: {}'.format(epoch, output))


Epoch 0 - loss: 313.978515625
Epoch 20 - loss: 0.08640945702791214
Epoch 40 - loss: 0.050383418798446655
Epoch 60 - loss: 0.029850633814930916
Epoch 80 - loss: 0.018024258315563202


In [32]:
print(list(net.parameters()), end='\n\n')

# Listar los atributos de los parámetros
for param in net.parameters():
    print(type(param.data), param.size())

[Parameter containing:
tensor([[ 1.7440],
        [-0.3228]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.1398, 0.7401], device='cuda:0', requires_grad=True), Parameter containing:
tensor([[ 1.6861, -0.4506]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.0624], device='cuda:0', requires_grad=True)]

<class 'torch.Tensor'> torch.Size([2, 1])
<class 'torch.Tensor'> torch.Size([2])
<class 'torch.Tensor'> torch.Size([1, 2])
<class 'torch.Tensor'> torch.Size([1])
